In [58]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img, load_img
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D, ZeroPadding2D, Activation, Add, AveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.optimizers import Adam, schedules
from tensorflow.keras import Input
from tensorflow.keras.regularizers import L2

import pandas as pd
import seaborn as sns
import numpy as np

In [59]:
def res_identity(x, filters): 
  #renet block where dimension doesnot change.
  #The skip connection is just simple identity conncection
  #we will have 3 blocks and then input will be added

    x_skip = x # this will be used for addition with the residual block 
    f1, f2 = filters

  #first block 
    x = Conv2D(f1, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=L2(0.001))(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

  #second block # bottleneck (but size kept same with padding)
    x = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=L2(0.001))(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

  # third block activation used after adding the input
    x = Conv2D(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=L2(0.001))(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

  # add the input 
    x = Add()([x, x_skip])
    x = Activation("relu")(x)
    return x

In [60]:
def res_conv(x, s, filters):
 
    x_skip = x
    f1, f2 = filters

  # first block
    x = Conv2D(f1, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_regularizer=L2(0.001))(x)
  # when s = 2 then it is like downsizing the feature map
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

  # second block
    x = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=L2(0.001))(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

  #third block
    x = Conv2D(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=L2(0.001))(x)
    x = BatchNormalization()(x)

  # shortcut 
    x_skip = Conv2D(f2, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_regularizer=L2(0.001))(x_skip)
    x_skip = BatchNormalization()(x_skip)

  # add 
    x = Add()([x, x_skip])
    x = Activation("relu")(x)
    return x

In [63]:
def resnet50():
    input_im = Input(shape=(100,100,3)) # cifar 10 images size
    x = ZeroPadding2D(padding=(3, 3))(input_im)

  # 1st stage
  # here we perform maxpooling, see the figure above

    x = Conv2D(64, kernel_size=(7, 7), strides=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPool2D((3, 3), strides=(2, 2))(x)

  #2nd stage 
  # frm here on only conv block and identity block, no pooling

    x = res_conv(x, s=1, filters=(64, 256))
    x = res_identity(x, filters=(64, 256))
    x = res_identity(x, filters=(64, 256))

  # 3rd stage

    x = res_conv(x, s=2, filters=(128, 512))
    x = res_identity(x, filters=(128, 512))
    x = res_identity(x, filters=(128, 512))
    x = res_identity(x, filters=(128, 512))

  # 4th stage

    x = res_conv(x, s=2, filters=(256, 1024))
    x = res_identity(x, filters=(256, 1024))
    x = res_identity(x, filters=(256, 1024))
    x = res_identity(x, filters=(256, 1024))
    x = res_identity(x, filters=(256, 1024))
    x = res_identity(x, filters=(256, 1024))

  # 5th stage

    x = res_conv(x, s=2, filters=(512, 2048))
    x = res_identity(x, filters=(512, 2048))
    x = res_identity(x, filters=(512, 2048))

  # ends with average pooling and dense connection

    x = AveragePooling2D((2, 2), padding='same')(x)

    x = Flatten()(x)
    x = Dense(3, activation='softmax', kernel_initializer='he_normal')(x) #multi-class

  # define the model 

    model = Model(inputs=input_im, outputs=x, name='Resnet50')

    return model


In [64]:
model = resnet50()

In [65]:
model.summary()

Model: "Resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_11 (ZeroPadding2 (None, 106, 106, 3)  0           input_13[0][0]                   
__________________________________________________________________________________________________
conv2d_175 (Conv2D)             (None, 50, 50, 64)   9472        zero_padding2d_11[0][0]          
__________________________________________________________________________________________________
batch_normalization_175 (BatchN (None, 50, 50, 64)   256         conv2d_175[0][0]                 
___________________________________________________________________________________________